In [15]:
# Libraries
import pandas as pd
import numpy as np

In [16]:
# Attempt #5: without priors

In [84]:
# Data
np.random.seed(123)
path_input = '/Users/annamenacher/Desktop/CDT/'
Lambda = pd.read_csv(path_input + 'LAMBDAn1500k50sim1.1', header = None)
beta = pd.read_csv(path_input + 'model_beta_bass_N1000.csv')
p=1803
v=5
n = 100
k = int(len(Lambda)/ p)
Lambda = Lambda.values.reshape(p,k)
beta = beta.values.reshape(k,1)
Z = np.random.normal(0,1, (n,k))
X = np.dot(Z, Lambda.T)
y = np.dot(Z, beta)

In [85]:
# Constants, dimensions, ...
eps = 0.1
n = 100
p = 1803
k = 50
v = 5

outer_prod_X = np.zeros(p * p).reshape(p,p)
sum_y_squared = 0
for i in range(n):
    outer_prod_X += np.dot(X[i,].reshape(p,1), X[i,].reshape(p,1).T)
    sum_y_squared += y[i,]**2
sum_y_squared = sum_y_squared.reshape(1,1)

In [86]:
# Initialisations
Lambda = np.random.normal(0,1, (p,k))
beta = np.random.normal(0,1, (k,1))
Sigma_x = np.diag(np.random.uniform(0,100,p))
sigma_y = 100

theta0 = [Lambda, beta, Sigma_x, sigma_y]

In [64]:
# Blockwise Inversion
def blockwise_inversion(A, B, C, D):
    '''
    Returns the four elements of a matrix after blockwise inversion with the following formula:
    https://en.wikipedia.org/wiki/Invertible_matrix#math_1

            Inputs:
                    A (pxp) (numpy array): Sigma_x + Lambda * Lambda.T
                    B (px1) (numpy array): Lambda * beta
                    C (1xp) (numpy array): (Lambda * beta).T
                    D (1x1) (numpy array): sigma_y + beta.T * beta

            Returns:
                    A, B, C, D ((pxp), (px1), (1xp), (1x1)) (numpy array): 
                    Matrices after blockwise inversion.
    '''
    
    A_Inv = np.linalg.pinv(A)
    AB = np.dot(A_Inv, B)
    CA = np.dot(C, A_Inv)
    DCAB = np.linalg.pinv(D - np.dot(CA, B))
    
    A = A_Inv + np.dot(np.dot(AB, DCAB), CA)
    B = - np.dot(AB, DCAB)
    C = - np.dot(DCAB, CA)
    D = DCAB
    
    return A, B, C, D

In [14]:
# EM-Algorithm 
num_iter = 0
diff = 1000
Lambda = theta0[0]
beta = theta0[1]
Sigma_x = theta0[2]
sigma_y = theta0[3]
Q = 0

while diff>eps:
    
    num_iter = num_iter + 1
    print(num_iter)
    Lambda_old = Lambda
    beta_old = beta
    Sigma_x_old = Sigma_x
    sigma_y_old = sigma_y
    Q_old = Q
    
    # E-step 
    A = (Sigma_x_old) + np.dot(Lambda_old, Lambda_old.T)
    B = np.dot(Lambda_old, beta_old)
    C = (np.dot(Lambda_old, beta_old)).T
    D = (sigma_y_old) + np.dot(beta_old.T, beta_old)

    A, B, C, D = blockwise_inversion(A, B, C, D)
    part1 = np.dot(Lambda_old.T, A) + np.dot(beta_old, C)
    part2 = np.dot(Lambda_old.T, B) + np.dot(beta_old, D)
    part3 = np.dot(part1, Lambda_old) + np.dot(part2, beta_old.T)

    sum_exp_xz = np.zeros(p * k).reshape(p,k)
    sum_exp_zx = np.zeros(p * k).reshape(k,p)
    sum_exp_yz = np.zeros(k).reshape(k,1)
    sum_exp_zz = np.zeros(k * k).reshape(k,k)

    for i in range(n):

        exp_z = np.dot(part1, X[i,].reshape(p,1)) + np.dot(part2, y[i,].reshape(1,1))
        sum_exp_xz += np.dot(X[i,].reshape(p,1), exp_z.T)
        sum_exp_zx += np.dot(exp_z, X[i,].reshape(p,1).T)
        sum_exp_yz += np.dot(exp_z, y[i,].reshape(1,1))
        sum_exp_zz += np.dot(exp_z, exp_z.T)

    sum_exp_zz += n * np.diag(np.ones(k)) - n * part3 
    
    # Pre-computed quantities 
    sum_exp_zz_inv = np.linalg.inv(sum_exp_zz)
    # M-step: Lambda
    Lambda = np.dot(sum_exp_xz, sum_exp_zz_inv)
    # M-step: beta 
    beta = np.dot(sum_exp_yz.T, sum_exp_zz_inv)
    beta = beta.T
    # M-step: Sigma_x
    prod1 = outer_prod_X - np.dot(sum_exp_xz, Lambda.T) - np.dot(Lambda, sum_exp_zx) + np.dot(np.dot(Lambda, sum_exp_zz), Lambda.T)
    prod2 = np.diag(np.diag(prod1))
    Sigma_x = (1/n) * prod2
    # M-step: sigma_y
    sigma_y = (1/n) * (sum_y_squared - 2* np.dot(beta.T, sum_exp_yz) + np.dot(np.dot(beta.T, sum_exp_zz), beta))
    
    # Calculate Q
    log_det_Sigma_x = 0
    for l in range(p):
        log_det_Sigma_x += np.log(Sigma_x[l,l])
    Sigma_x_inv = np.linalg.inv(Sigma_x)
    Q = -(n/2) * log_det_Sigma_x - (1/2) * np.trace(np.dot(Sigma_x_inv, prod1)) - (n/2)* np.log(sigma_y) - (1/2) * n * sigma_y - (1/2) * np.trace(sum_exp_zz)
    print(Q)
    diff = abs(Q-Q_old)
    
theta = [None] * 4
theta[0] = Lambda
theta[1] = beta
theta[2] = Sigma_x
theta[3] = sigma_y 

1


/Users/annamenacher/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:63: RuntimeWarning: divide by zero encountered in log


LinAlgError: Singular matrix

In [7]:
# note: singular matrix due to zero diagonal elements in the matrix Sigma_x 
# -> hence inversion is impossible

In [ ]:
# Habib's derivation of the EM algorithm 

In [82]:
# EM-Algorithm 
num_iter = 0
diff = 1000
Lambda = theta0[0]
beta = theta0[1]
Sigma_x = theta0[2]
sigma_y = theta0[3]
Q = 0

while diff>eps:
    
    num_iter = num_iter + 1
    print(num_iter)
    Lambda_old = Lambda
    beta_old = beta
    Sigma_x_old = Sigma_x
    sigma_y_old = sigma_y
    Q_old = Q
    
    var_z = np.linalg.inv(np.diag(np.ones(k)) - np.dot(beta_old, beta_old.T) * (1/sigma_y_old) - \
                          np.dot(np.dot(Lambda_old.T, np.linalg.inv(Sigma_x_old)), Lambda_old))
                                 
    exp_z = np.empty((n,k))
    for i in range(n):
        exp_z2 = np.dot(var_z, (np.dot(np.dot(Lambda_old.T, np.linalg.inv(Sigma_x_old)), X[i,].reshape(p,1)) +\
                                   np.dot(beta_old,y[i,].reshape(1,1)).reshape(k,1)))
        exp_z[i,] = exp_z2.T
    
    # Pre-computed quantities 
    exp_zz_inv = np.linalg.inv(var_z)
    # M-step: Lambda
    Lambda = np.dot(exp_zz_inv,np.dot(exp_z.T, X)).T
    # M-step: beta 
    beta = np.dot(exp_zz_inv, np.dot(exp_z.T, y))
    #beta = beta.T
    # M-step: Sigma_x
    prod1 = outer_prod_X - np.dot(np.dot(X.T,exp_z), Lambda.T) - np.dot(Lambda, np.dot(exp_z.T,X)) + np.dot(np.dot(Lambda, (var_z)), Lambda.T)
    prod2 = np.diag(np.diag(prod1))
    Sigma_x = (1/n) * prod2
    # M-step: sigma_y
    sigma_y = (1/n) * (sum_y_squared - 2* np.dot(beta.T, np.dot(exp_z.T,y)) + np.dot(np.dot(beta.T, (var_z)), beta))
    
    # Calculate Q
    log_det_Sigma_x = 0
    for l in range(p):
        log_det_Sigma_x += np.log(Sigma_x[l,l])
    Sigma_x_inv = np.linalg.inv(Sigma_x)
    Q = -(n/2) * log_det_Sigma_x - (1/2) * np.trace(np.dot(Sigma_x_inv, prod1)) - (n/2)* np.log(sigma_y) - (1/2) * n * sigma_y - (1/2) * np.trace(var_z)
    print(Q)
    diff = abs(Q-Q_old)
    
theta = [None] * 4
theta[0] = Lambda
theta[1] = beta
theta[2] = Sigma_x
theta[3] = sigma_y 

1


/Users/annamenacher/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:46: RuntimeWarning: divide by zero encountered in log


LinAlgError: Singular matrix

In [83]:
np.count_nonzero(np.diag(Sigma_x))

1795

In [87]:
Sigma_x

array([[99.19148634,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        , 69.32649681,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        , 84.83311083, ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [ 0.        ,  0.        ,  0.        , ..., 97.93850958,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
        57.50898721,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        , 21.96922762]])